In [ ]:
!pip install mysql-connector-python


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.4/27.4 MB 65.2 MB/s eta 0:00:00


In [ ]:
import mysql.connector
import pandas as pd
# Replace the placeholders with your own values
connection_name = 'c23df02-diskusai:asia-southeast2:diskusaidataset'
ip_address = '34.101.70.233'
db_name = 'diskusaidataset'
user = 'root'
password = 'df02'

# Create the connection string
connection_string = f"mysql+mysqlconnector://{user}:{password}@{ip_address}/{db_name}?unix_socket=/cloudsql/{connection_name}"

# Establish the connection
cnx = mysql.connector.connect(user=user, password=password, host=ip_address, database=db_name)


In [ ]:
# Replace 'your_table_name' with the actual name of your table
query = "SELECT * FROM diskusaidataset"

# Execute the query
cursor = cnx.cursor()
cursor.execute(query)

# Fetch all the results
data = cursor.fetchall()

# Convert the data into a Pandas DataFrame
df = pd.DataFrame(data, columns=cursor.column_names)

# Close the cursor and connection
cursor.close()
cnx.close()


In [ ]:
df['question'] = df['question'].str.replace('<p>', '')
df['question'] = df['question'].str.replace('<h2>', '')
df['question'] = df['question'].str.replace('</h2>', '')
df['question'] = df['question'].str.replace('</p>', '')
df['question'] = df['question'].str.replace('<strong>', '')
df['question'] = df['question'].str.replace('</strong>', '')
df = df[df['question'].str.contains('<img src=')==False]
df=df.reset_index(drop=True)

In [ ]:
df.head()

,id,course_id,module_name,tutorial_id,discussion_title,question,solved,tags
0,0,0,module_name,0,discussion_title,question,0,tags
1,8527,123,Mekanisme Belajar,5955,menu DropDwon Tidak Muncul,"Halo rekan-rekan dicodingMohon bantuannya,Menu...",1,[dropdown-tidak-muncu]
2,8591,123,Mekanisme Belajar,5955,Responsibilitas Submission Kurang Jelas,"Dalam submission tertera 4 column, akan tetapi...",1,"[responsive,web]"
3,8609,123,Mekanisme Belajar,5955,Cara penulisan array dan object yg bener yg ma...,"Teksnya:""Array adalah tipe data yang ditulis d...",1,"[tipe-data,data-type,typo]"
4,8652,123,Mekanisme Belajar,5955,Bagaimana cara menghasilkan output yang hanya ...,"Saya punya kode seperti berikut :var pesan = ""...",1,"[web,string]"


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8833 entries, 0 to 8832
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                8833 non-null   int64 
 1   course_id         8833 non-null   int64 
 2   module_name       8833 non-null   object
 3   tutorial_id       8833 non-null   int64 
 4   discussion_title  8833 non-null   object
 5   question          8833 non-null   object
 6   solved            8833 non-null   int64 
 7   tags              8833 non-null   object
dtypes: int64(4), object(4)
memory usage: 552.2+ KB


In [ ]:
df= df.sample(n=883,random_state=7)

In [ ]:
!pip install keras-tuner
import tensorflow as tf
from keras_tuner import BayesianOptimization
from sklearn.model_selection import train_test_split
import re
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 5.6 MB/s eta 0:00:00


In [ ]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('all')

def clean_text(text):
    text = text.lower()
    text = re.sub('[^\w\s]', '', text)
    text = re.sub('\'s', '', text)
    text = re.sub('\\r\\n', '', text)
    text = re.sub(r'\b-\b', '', text)
    
    tokens = word_tokenize(text)
    
    # Menghapus stop word
    stop_words = set(stopwords.words('indonesian')) # Ganti dengan bahasa yang sesuai
    tokens = [token for token in tokens if token not in stop_words]
    
    # Stemming menggunakan Porter Stemmer
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    # Menggabungkan token menjadi kalimat kembali
    tokens = ' '.join(tokens)
   
    return tokens

df['clean_question'] = df['question'].apply(clean_text)
tokens = df['clean_question'].str.split()
tokens = [word for sublist in tokens for word in sublist]

print(tokens[:50])


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

['link', 'download', 'httpsgithubcomdicodingacademya315webpemulalabsrawsharedfilesbookshelfappszip', 'function', 'minimala', 'b', 'nbsp', 'if', 'a', 'lt', 'b', 'nbsp', 'nbsp', 'return', 'anbsp', 'else', 'if', 'a', 'gt', 'b', 'nbsp', 'nbsp', 'return', 'bnbsp', 'else', 'if', 'a', 'b', 'nbsp', 'nbsp', 'return', 'anbsp', 'consolelogminimal3', '2brlet', 'powerresult', 'power7', '3function', 'powera', 'b', 'nbsp', 'nbspnbspreturn', 'a', 'bconsolelogpowerresult', 'halo', 'kak', 'catch', 'async', 'await', 'modul', 'handle']


In [ ]:
STOPWORDS1 = set(stopwords.words('indonesian'))
STOPWORDS2 = set(stopwords.words('english'))
MIN_WORDS = 4
MAX_WORDS = 200

PATTERN_S = re.compile("\'s")  # matches `'s` from text  
PATTERN_RN = re.compile("\\r\\n") #matches `\r` and `\n`
PATTERN_PUNC = re.compile(r"[^\w\s]") # matches all non 0-9 A-z whitespace 

def clean_text(text):
    """
    Series of cleaning. String to lower case, remove non words characters and numbers.
        text (str): input text
    return (str): modified initial text
    """
    text = text.lower()  # lowercase text
    text = re.sub(PATTERN_S, ' ', text)
    text = re.sub(PATTERN_RN, ' ', text)
    text = re.sub(PATTERN_PUNC, ' ', text)
    return text

def tokenizer1(sentence, min_words=MIN_WORDS, max_words=MAX_WORDS, stopwords=STOPWORDS1, lemmatize=True):
    """
    Lemmatize, tokenize, crop and remove stop words.
    """
    stemmer = WordNetLemmatizer()
    tokens1 = [stemmer.lemmatize(w) for w in word_tokenize(sentence)]
    tokens1 = ' '.join(tokens1)
    return tokens1    

def tokenizer2(sentence, min_words=MIN_WORDS, max_words=MAX_WORDS, stopwords=STOPWORDS2, lemmatize=True):
    """
    Lemmatize, tokenize, crop and remove stop words.
    """
    stemmer = WordNetLemmatizer()
    tokens2 = [stemmer.lemmatize(w) for w in word_tokenize(sentence)]
    tokens2 = ' '.join(tokens2)
    return tokens2    

def clean_sentences(df):
    """
    Remove irrelavant characters (in new column clean_sentence).
    Lemmatize, tokenize words into list of words (in new column tok_lem_sentence).
    """
    print('Cleaning sentences...')
    df['question'] = df['question'].apply(clean_text)
    df['question'] = df['question'].apply(
        lambda x: tokenizer1(x, min_words=MIN_WORDS, max_words=MAX_WORDS, stopwords=STOPWORDS1, lemmatize=True))
    df['question'] = df['question'].apply(
        lambda x: tokenizer2(x, min_words=MIN_WORDS, max_words=MAX_WORDS, stopwords=STOPWORDS2, lemmatize=True))
    return df
    
df = clean_sentences(df)
tokens = df['question'].str.split()
tokens = [word for sublist in tokens for word in sublist]

Cleaning sentences...


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# length = 50 +1
# lines = []

# for i in range(length, len(tokens)):
#     seq = tokens[i-length:i]
#     line = ' '.join(seq)
#     lines.append(line)
#     if i > 200000:
#         break
# print(len(lines))


tokenizer = Tokenizer(oov_token='<oov>') # For those words which are not found in word_index
tokenizer.fit_on_texts(tokens)
total_words = len(tokenizer.word_index) + 1

In [ ]:
input_sequences = []
for line in tokens:
    token_list = tokenizer.texts_to_sequences([line])[0]
    #print(token_list)
    
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# print(input_sequences)
print("Total input sequences: ", len(input_sequences))

# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(lines)
# sequences = tokenizer.texts_to_sequences(lines)

Total input sequences:  181


In [ ]:
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [ ]:
import tensorflow as tf
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]
ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [ ]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.optimizers import Adam
model = Sequential()
model.add(Embedding(total_words, 32, input_length=max_sequence_len-1))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(128,kernel_regularizer=tf.keras.regularizers.l2(0.001),recurrent_regularizer=tf.keras.regularizers.l2(0.001))))
model.add(Dropout(0.5))
model.add(Dense(total_words, activation='softmax',kernel_regularizer=tf.keras.regularizers.l2(0.001)))
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=10000,
    decay_rate=0.95)
optimizers = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
model.compile(loss='categorical_crossentropy', optimizer=optimizers, metrics=['accuracy'])
history = model.fit(xs, ys, epochs=1000, validation_split=0.02, verbose=1)
#print model.summary()
print(model)

Epoch 1/1000
6/6 [==============================] - 16s 873ms/step - loss: 8.6552 - accuracy: 0.0847 - val_loss: 7.7933 - val_accuracy: 0.0000e+00
Epoch 2/1000
6/6 [==============================] - 2s 213ms/step - loss: 5.7432 - accuracy: 0.1186 - val_loss: 5.2834 - val_accuracy: 0.0000e+00
Epoch 3/1000
6/6 [==============================] - 1s 228ms/step - loss: 4.6058 - accuracy: 0.0960 - val_loss: 4.6337 - val_accuracy: 0.0000e+00
Epoch 4/1000
6/6 [==============================] - 1s 121ms/step - loss: 4.3416 - accuracy: 0.1243 - val_loss: 4.2688 - val_accuracy: 0.0000e+00
Epoch 5/1000
6/6 [==============================] - 0s 91ms/step - loss: 4.0373 - accuracy: 0.0960 - val_loss: 4.2031 - val_accuracy: 0.0000e+00
Epoch 6/1000
6/6 [==============================] - 1s 110ms/step - loss: 3.7303 - accuracy: 0.0847 - val_loss: 4.2186 - val_accuracy: 0.0000e+00
Epoch 7/1000
6/6 [==============================] - 1s 169ms/step - loss: 3.5634 - accuracy: 0.1299 - val_loss: 4.3200 - val

In [ ]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.show()

In [ ]:
token_list = tokenizer.texts_to_sequences(['java'])[0]
token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
predicted = model.predict(token_list)
predicted = np.argmax(predicted,axis=1)
for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break

1/1 [==============================] - 1s 667ms/step


In [ ]:
print(output_word)

apps


In [ ]:
seed_text = "akun"
seed_text_sentence=seed_text
next_words = 10
  
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict(token_list)
    predicted = np.argmax(predicted,axis=1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text_sentence += " " + output_word
    seed_text = output_word
print(seed_text_sentence)

1/1 [==============================] - 0s 30ms/step
akun 3kut6czozt5qnigs6s apps apps apps apps apps apps apps apps apps


In [ ]:
seed_text = "jquery"
next_words = 3
  
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict(token_list)
    predicted = np.argmax(predicted,axis=1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word
print(seed_text)

1/1 [==============================] - 0s 31ms/step
jquery apps apps apps


In [ ]:
seed_text = "dropdown"
next_words = 1
  
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict(token_list)
    predicted = np.argmax(predicted,axis=1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word
print(seed_text)

1/1 [==============================] - 0s 33ms/step
dropdown apps


In [ ]:
seed_text = "java"
next_words = 2
  
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict(token_list)
    predicted = np.argmax(predicted,axis=1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word
print(seed_text)

1/1 [==============================] - 0s 36ms/step
java apps apps
